Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score


1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

2. Подготовьте датасет к обучению моделей:
* Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.

In [ ]:
data.info()

In [ ]:
data_encoded = pd.get_dummies(data, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])
data_encoded.head()

In [ ]:
data_encoded.columns

3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [ ]:
X = data_encoded.drop('HeartDisease', axis=1)
y = data_encoded['HeartDisease']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

4. Обучите модель логистической регрессии с параметрами по умолчанию.

In [ ]:
# Алгоритм логистической регрессии не отработал за max_iter=100 по умолчанию

model = LogisticRegression(max_iter=5000, random_state=42) 
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred

5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])

In [ ]:
scoring_metrics = ['accuracy', 'recall', 'precision', 'f1']

In [ ]:
cv_results = cross_validate(
    model,
    X=X,
    y=y,
    cv=10,
    scoring=scoring_metrics,
    n_jobs=-1,
    return_train_score=True
)

In [ ]:
print(f'''Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: {cv_results['test_accuracy'].mean():.4f},
    Train Accuracy: {cv_results['train_accuracy'].mean():.4f},

    Test Recall: {cv_results['test_recall'].mean():.4f},
    Train Recall: {cv_results['train_recall'].mean():.4f},

    Test Precision: {cv_results['test_precision'].mean():.4f},
    Train Precision: {cv_results['train_precision'].mean():.4f},

    Test F1: {cv_results['test_f1'].mean():.4f},
    Train F1: {cv_results['train_f1'].mean():.4f}
''')

6. Оптимизируйте 3-4 параметра модели:
* Используйте GridSearchCV.
* Используйте RandomizedSearchCV.
* Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.
* Повторите п. 5 после каждого итогового изменения параметров.

In [ ]:
# GridSearchCV для LogisticRegression

param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['saga']
}

grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Лучшие параметры GridSearchCV LogisticRegression:")
print(grid_search.best_params_)

best_model = grid_search.best_estimator_

scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(
    best_model,
    X,
    y,
    cv=10,
    scoring=scoring_metrics,
    n_jobs=-1,
    return_train_score=True
)

print(f'''Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: {cv_results['test_accuracy'].mean():.4f},
    Train Accuracy: {cv_results['train_accuracy'].mean():.4f},

    Test Recall: {cv_results['test_recall'].mean():.4f},
    Train Recall: {cv_results['train_recall'].mean():.4f},

    Test Precision: {cv_results['test_precision'].mean():.4f},
    Train Precision: {cv_results['train_precision'].mean():.4f},

    Test F1: {cv_results['test_f1'].mean():.4f},
    Train F1: {cv_results['train_f1'].mean():.4f}
''')

In [ ]:
# RandomizedSearchCV для LogisticRegression

param_randomsearch = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['saga']
}

random_search = RandomizedSearchCV(
    model,
    param_distributions=param_randomsearch,
    n_iter=12,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42 
)

random_search.fit(X_train, y_train)

print("Лучшие параметры RandomizedSearchCV LogisticRegression:")
print(random_search.best_params_)

best_model = random_search.best_estimator_

scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'recall': make_scorer(recall_score),
    'precision': make_scorer(precision_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(
    best_model,
    X,
    y,
    cv=10,
    scoring=scoring_metrics,
    n_jobs=-1,
    return_train_score=True
)

print(f'''Среднее по метрикам кросс-валидации LogisticRegression
    Test Accuracy: {cv_results['test_accuracy'].mean():.4f},
    Train Accuracy: {cv_results['train_accuracy'].mean():.4f},

    Test Recall: {cv_results['test_recall'].mean():.4f},
    Train Recall: {cv_results['train_recall'].mean():.4f},

    Test Precision: {cv_results['test_precision'].mean():.4f},
    Train Precision: {cv_results['train_precision'].mean():.4f},

    Test F1: {cv_results['test_f1'].mean():.4f},
    Train F1: {cv_results['train_f1'].mean():.4f}
''')

7. Сформулируйте выводы по проделанной работе:
* Сравните метрики построенных моделей.
* Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».